**Actividad 01**

Alumno: Huillca Herrera Patrick David

Realizaremos una consulta a la apinews para ver noticias recientes para evaluar los datos definimos cuatro indicadores de calidad
- Completitud de que todas las noticias tengan al menos un autor
- Completitud que las noticias tengan al menos una fuente
- Noticias recientes: Que nos presente noticias de a lo mas 3 días de antiguedad
- Que tengan una descripción valida: Que las noticias como mínimo tengan una descripción de 2 palabras

In [1]:

import requests
import pandas as pd
from datetime import datetime, timedelta

# Tu clave API de NewsAPI

# URL de la API de NewsAPI para buscar noticias sobre Perú
url = f'https://newsapi.org/v2/everything?q=Perú&apiKey={api_key}'

# Realizar la solicitud GET
response = requests.get(url)

# Verificamos si la solicitud fue exitosa
if response.status_code == 200:
    news_data = response.json()

    # Fecha actual y fecha límite para las noticias (3 días atrás)
    today = datetime.today()
    three_days_ago = today - timedelta(days=3)

    # Variables para las métricas
    total_articles = len(news_data['articles'])
    articles_with_author = 0
    articles_with_source = 0
    articles_recent = 0
    articles_with_description = 0

    # Lista para almacenar las noticias filtradas
    filtered_articles = []

    # Filtrar noticias y calcular métricas
    for article in news_data['articles']:
        published_at = article.get('publishedAt', '')
        description = article.get('description', '')
        author = article.get('author', None)
        source = article.get('source', {}).get('name', None)

        # Fecha de publicación
        published_date = datetime.strptime(published_at, '%Y-%m-%dT%H:%M:%SZ') if published_at else None

        # Filtrar noticias recientes (hasta 3 días)
        if published_date and published_date >= three_days_ago:
            articles_recent += 1

        # Verificar si tiene autor
        if author:
            articles_with_author += 1

        # Verificar si tiene fuente
        if source:
            articles_with_source += 1

        # Verificar si la descripción tiene al menos 2 palabras
        if len(description.split()) >= 2:
            articles_with_description += 1

        # Agregar la noticia al listado de noticias filtradas
        filtered_articles.append({
            'Title': article['title'],
            'Description': description,
            'Source': source,
            'Author': author if author else 'Unknown',
            'PublishedAt': published_at,
            'URL': article['url']
        })

    # Calcular métricas
    percent_with_author = (articles_with_author / total_articles) * 100 if total_articles > 0 else 0
    percent_with_source = (articles_with_source / total_articles) * 100 if total_articles > 0 else 0
    percent_recent = (articles_recent / total_articles) * 100 if total_articles > 0 else 0
    percent_with_description = (articles_with_description / total_articles) * 100 if total_articles > 0 else 0

    # Mostrar métricas
    print(f'Métricas de Noticias:')
    print(f'Total de noticias obtenidas: {total_articles}')
    print(f'Noticias con al menos un autor: {articles_with_author} ({percent_with_author:.2f}%)')
    print(f'Noticias con fuente: {articles_with_source} ({percent_with_source:.2f}%)')
    print(f'Noticias con antigüedad de máximo 3 días: {articles_recent} ({percent_recent:.2f}%)')
    print(f'Noticias con descripción de al menos 2 palabras: {articles_with_description} ({percent_with_description:.2f}%)')

    # Usar pandas para crear el DataFrame y exportar a CSV
    news_df = pd.DataFrame(filtered_articles)

    # Exportar el DataFrame a un archivo CSV
    news_df.to_excel('noticias_peru.xlsx', encoding='utf-8')

    # Exportar las métricas a un archivo CSV con pandas
    metrics = {
        'Metric': ['Total Articles', 'Articles with Author', 'Articles with Source',
                   'Articles with Recent Date (< 3 days)', 'Articles with Description (2+ words)',
                   'Percentage with Author', 'Percentage with Source',
                   'Percentage with Recent Date (< 3 days)', 'Percentage with Description (2+ words)'],
        'Value': [total_articles, articles_with_author, articles_with_source,
                  articles_recent, articles_with_description,
                  f'{percent_with_author:.2f}%', f'{percent_with_source:.2f}%',
                  f'{percent_recent:.2f}%', f'{percent_with_description:.2f}%']
    }

    metrics_df = pd.DataFrame(metrics)

    # Exportar las métricas a CSV
    metrics_df.to_csv('noticias_peru_metrics.xlsx', index=False, encoding='utf-8')

    print("Datos exportados a 'noticias_peru_listado.csv' y métricas a 'noticias_peru_metrics.csv'.")
else:
    print(f'Error al obtener las noticias: {response.status_code}')

Métricas de Noticias:
Total de noticias obtenidas: 99
Noticias con al menos un autor: 97 (97.98%)
Noticias con fuente: 99 (100.00%)
Noticias con antigüedad de máximo 3 días: 10 (10.10%)
Noticias con descripción de al menos 2 palabras: 94 (94.95%)
Datos exportados a 'noticias_peru_listado.csv' y métricas a 'noticias_peru_metrics.csv'.


In [2]:
news_df

,Title,Description,Source,Author,PublishedAt,URL
0,Cerveza psicodélica: el «truco» que usaban en ...,En 2022 se publicó un estudio en el que se hab...,Hipertextual,Azucena Martín,2025-03-25T11:12:58Z,http://hipertextual.com/2025/03/cerveza-psicod...
1,EN DIRECTO: Presentación oficial de la Nintend...,"Hoy es el día, Nintendo lo tiene todo preparad...",Hipertextual,Alberto Martín,2025-04-02T12:33:30Z,http://hipertextual.com/2025/04/directo-ninten...
2,El adiós latinoamericano de Telefónica: Murtra...,Telefónica está ejecutando una salida acelerad...,Xataka.com,Javier Lacort,2025-03-13T07:00:42Z,https://www.xataka.com/empresas-y-economia/adi...
3,Nintendo Switch 2: sigue en directo la present...,Llegó el día. Los nintenderos estamos de celeb...,Xataka.com,Ricardo Aguilar,2025-04-02T08:36:00Z,https://www.xataka.com/nuevo/nintendo-switch-2...
4,Descubrimiento arqueológico en Perú desafía lo...,Un mural de 1.500 años ha salido a la luz en P...,Gizmodo.com,Lucas Handley,2025-03-09T15:41:47Z,https://es.gizmodo.com/descubrimiento-arqueolo...
...,...,...,...,...,...,...
94,Cuáles son los partidos de Venezuela en esta v...,A pesar del mal momento y la gran cantidad de ...,La Nacion,LA NACION,2025-03-20T12:32:00Z,https://www.lanacion.com.ar/estados-unidos/cua...
95,La Princesa Leonor estudia las cartas de naveg...,El Juan Sebastián Elcano se encuentra en el es...,El Mundo,Marina Pina,2025-03-25T16:49:25Z,https://www.elmundo.es/espana/2025/03/25/67e2d...
96,Cincuenta años en la escena del metal y se vue...,En el mundo de internet es muy común que las c...,Trendencias.com,Joel Calata,2025-03-06T16:00:13Z,https://www.trendencias.com/mexico/cincuenta-a...
97,Advertencia por cantos racistas y discriminato...,"Infortunadamente, en cada partido de la Selecc...",Marca,marca.com,2025-03-24T21:12:56Z,https://www.marca.com/co/2025/03/24/67e1c5c826...


Realizaremos una consulta a la apiweather para ver la temperatura de 4 departamentos del país (mínima y máxima del dia) para evaluar los datos definimos cuatro indicadores de calidad
- Completitud que esten las temperaturas de los 4 departamentos a la fecha de hoy
- Que la temperatura maxima no sea menor a la temperatura mínima (coherencia de la data)
- Que los valores de termperatura sean de tipo float para su uso y comparación con otras temperaturas (que no sean valor string)
- Que todos los registros tengan los datos de temperatura mínima y máxima completa (que no falte ninguno de esos datos)

In [3]:
import requests
import pandas as pd

# Tu clave API de WeatherAPI (cámbiala por la tuya)
# NO SE ADJUNTA API KEY por indicación del profesor
# Coordenadas de las ciudades en Perú
cities_coordinates = {
    'Lima': {'lat': -12.0464, 'lon': -77.0428},
    'Arequipa': {'lat': -16.4090, 'lon': -71.5375},
    'Cusco': {'lat': -13.5320, 'lon': -71.9675},
    'Trujillo': {'lat': -8.1103, 'lon': -79.0193}
}

# Lista para almacenar los registros
records = []

# Realizamos la solicitud para cada ciudad
for city, coords in cities_coordinates.items():
    # URL de la API de WeatherAPI para obtener los datos climáticos (temperatura mínima y máxima)
    url = f'http://api.weatherapi.com/v1/forecast.json?key={api_key}&q={coords["lat"]},{coords["lon"]}&days=1'

    # Realizar la solicitud GET
    response = requests.get(url)

    # Verificamos si la solicitud fue exitosa
    if response.status_code == 200:
        weather_data = response.json()

        # Extraemos los registros de temperatura máxima y mínima por cada día
        if 'forecast' in weather_data and 'forecastday' in weather_data['forecast']:
            for daily_data in weather_data['forecast']['forecastday']:
                temp_max = daily_data['day']['maxtemp_c']
                temp_min = daily_data['day']['mintemp_c']
                date = daily_data['date']

                # Comprobamos si alguna de las temperaturas es None o vacía
                if temp_max is None or temp_min is None:
                    continue  # Saltamos el registro si hay valores vacíos en alguna de las temperaturas

                records.append({
                    'City': city,
                    'Date': date,
                    'Temp_Max': temp_max,
                    'Temp_Min': temp_min
                })
    else:
        print(f'Error al obtener los datos del clima para {city}: {response.status_code}')

# Crear un DataFrame de Pandas con los registros de las temperaturas
df = pd.DataFrame(records)

# Mostrar el tipo de datos de las columnas
print("Tipos de datos de las columnas:")
print(df.dtypes)

# Total de registros
total_records = len(df)

# 1. Número de filas donde no haya una temperatura máxima o mínima
missing_temps = df[df['Temp_Max'].isnull() | df['Temp_Min'].isnull()]
missing_temps_count = len(missing_temps)
missing_temps_percentage = (missing_temps_count / total_records) * 100

# 2. Número de filas donde la temperatura máxima sea menor que la mínima
max_less_than_min = len(df[df['Temp_Max'] < df['Temp_Min']])
max_less_than_min_percentage = (max_less_than_min / total_records) * 100

# 3. Verificar que no haya filas vacías
empty_rows = df.isnull().all(axis=1)  # Identifica filas completamente vacías
empty_rows_count = empty_rows.sum()
empty_rows_percentage = (empty_rows_count / total_records) * 100

# 4. Verificar que las temperaturas son de tipo float (y asegurar que no haya otros tipos)
temp_max_is_float = df['Temp_Max'].apply(lambda x: isinstance(x, float)).sum()
temp_min_is_float = df['Temp_Min'].apply(lambda x: isinstance(x, float)).sum()

# Mostrar los resultados de la validación
print(f'\nTotal de registros: {total_records}')
print(f'\nNúmero de filas donde falta Temp_Max o Temp_Min: {missing_temps_count} ({missing_temps_percentage:.2f}%)')
print(f'Número de filas donde Temp_Max < Temp_Min: {max_less_than_min} ({max_less_than_min_percentage:.2f}%)')
print(f'Número de filas vacías: {empty_rows_count} ({empty_rows_percentage:.2f}%)')
print(f'Número de registros donde Temp_Max es float: {temp_max_is_float} ({(temp_max_is_float / total_records) * 100:.2f}%)')
print(f'Número de registros donde Temp_Min es float: {temp_min_is_float} ({(temp_min_is_float / total_records) * 100:.2f}%)')

# Validación: Verificar que no haya filas vacías o nulas
df_clean = df.dropna(how='all')  # Elimina las filas completamente vacías

# Exportar el DataFrame limpio a un archivo CSV con los datos climáticos
df_clean.to_csv('clima_peru_weatherapi_limpio.csv', index=False, encoding='utf-8')
print("\nLos datos climáticos de las ciudades en Perú han sido exportados a 'clima_peru_weatherapi_limpio.csv'.")

Tipos de datos de las columnas:
City         object
Date         object
Temp_Max    float64
Temp_Min    float64
dtype: object

Total de registros: 4

Número de filas donde falta Temp_Max o Temp_Min: 0 (0.00%)
Número de filas donde Temp_Max < Temp_Min: 0 (0.00%)
Número de filas vacías: 0 (0.00%)
Número de registros donde Temp_Max es float: 4 (100.00%)
Número de registros donde Temp_Min es float: 4 (100.00%)

Los datos climáticos de las ciudades en Perú han sido exportados a 'clima_peru_weatherapi_limpio.csv'.


In [4]:
df

,City,Date,Temp_Max,Temp_Min
0,Lima,2025-04-04,23.0,19.3
1,Arequipa,2025-04-04,20.0,9.7
2,Cusco,2025-04-04,16.1,5.8
3,Trujillo,2025-04-04,23.7,20.0
